In [1]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os, sys, time
import tensorflow as tf
from tensorflow import keras

在进行模型建设之前，一般应该完成如下三个操作:
- 词典构建(id - &gt; word, word -&gt; id)
- 数据变换 (sentence -&gt; id)
- 训练集和测试集的构建(abcd -> bcd&lt;eos&gt;)

## **3.1 数据集读取**

In [2]:
input_filepath = './data/poem/tang_poems_7.txt'

with open(input_filepath, 'r', encoding='utf-8') as f:
    text = ''
    for s in f.readlines():
        if '□' not in s:
            text = text + s
print(len(text))
print(text[0:85])

1243210
采玉采玉须水碧，琢作步摇徒好色。
傍水野禽通体白，饤盘山菓半边红。
宜秋下邑摧凶丑，当锋入阵宋中丞。
吾为子起歌都护，酒阑插剑肝胆露。
南北东西九千里，除兄与弟更无人。



## **3.2 词表操作**
### **3.2.1 生成词表**

In [3]:
vocab = sorted(set(text))
print(len(vocab))
print(vocab[10:100])

6359
['䝙', '䯄', '䲡', '䴔', '䴖', '䴙', '一', '丁', '七', '万', '丈', '三', '上', '下', '不', '与', '丐', '丑', '专', '且', '丕', '世', '丘', '丙', '业', '丛', '东', '丝', '丞', '两', '严', '丧', '个', '丫', '中', '丰', '丱', '串', '临', '丸', '丹', '为', '主', '丽', '举', '乃', '久', '么', '义', '之', '乌', '乍', '乎', '乏', '乐', '乔', '乖', '乘', '乙', '九', '乞', '也', '习', '乡', '书', '买', '乱', '乳', '乾', '了', '予', '争', '事', '二', '于', '亏', '云', '互', '五', '井', '亘', '亚', '些', '亡', '亢', '交', '亥', '亦', '产', '亨']


In [4]:
char2idx = {char : idx for idx, char in enumerate(vocab)}
print(char2idx['一'])

16


In [5]:
vocab[:10]

['\n', '。', '㔉', '㧑', '㶉', '䃅', '䌷', '䍀', '䗖', '䜩']

In [6]:
idx2_char = np.array(vocab)

## **3.2.2 将数据集转换为数字**

In [7]:
text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int[0:17])
print(text[0:17])

[5497 3177 5497 3177 5857 2597 3565 6270 3222  188 2537 1960 1524 1009
 4334    1    0]
采玉采玉须水碧，琢作步摇徒好色。



In [8]:
def split_input_target(id_text):
    '''
    abcde -> abcd, bcde, 这里是给定生成，如果需要对联的话，需要id_text[0:7],[7:15]
    '''
    return id_text[0: 7], id_text[8:-2]

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 16 # 7 + 1 + 7 + 1
seq_dataset = char_dataset.batch(seq_length + 1, drop_remainder = True) # 加一是为了产出\n, 这个batch是为了生成一个句子
for ch_id in char_dataset.take(2):
    print(ch_id, idx2_char[ch_id.numpy()])

for seq_id in seq_dataset.take(2):
    print(seq_id)
    print(''.join(idx2_char[seq_id.numpy()]))

tf.Tensor(5497, shape=(), dtype=int32) 采
tf.Tensor(3177, shape=(), dtype=int32) 玉
tf.Tensor(
[5497 3177 5497 3177 5857 2597 3565 6270 3222  188 2537 1960 1524 1009
 4334    1    0], shape=(17,), dtype=int32)
采玉采玉须水碧，琢作步摇徒好色。

tf.Tensor(
[ 288 2597 5501 3640 5342  182 3395 6270 5916 3431 1247 4479  523 5286
 3929    1    0], shape=(17,), dtype=int32)
傍水野禽通体白，饤盘山菓半边红。



In [9]:
#调用seq生成x和y
seq_dataset = seq_dataset.map(split_input_target)

for item_input, item_output in seq_dataset.take(2):
    print(item_input, ''.join(idx2_char[item_input.numpy()]))
    print(item_output, repr(''.join(idx2_char[item_output.numpy()])))

tf.Tensor([5497 3177 5497 3177 5857 2597 3565], shape=(7,), dtype=int32) 采玉采玉须水碧
tf.Tensor([3222  188 2537 1960 1524 1009 4334], shape=(7,), dtype=int32) '琢作步摇徒好色'
tf.Tensor([ 288 2597 5501 3640 5342  182 3395], shape=(7,), dtype=int32) 傍水野禽通体白
tf.Tensor([5916 3431 1247 4479  523 5286 3929], shape=(7,), dtype=int32) '饤盘山菓半边红'


## **3.3 模型构建**

In [10]:
batch_size = 64
buffer_size = 1000
seq_dataset = seq_dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [11]:

vocab_size = len(vocab)
embedding_dim = 128
rnn_units = 512

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        keras.layers.Bidirectional(keras.layers.LSTM(units=rnn_units, return_sequences=True, 
                                                     recurrent_initializer = 'glorot_uniform')),
        keras.layers.Bidirectional(keras.layers.LSTM(units=rnn_units, return_sequences=True, 
                                                     recurrent_initializer = 'glorot_uniform')),
        keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(vocab_size=vocab_size,
                    embedding_dim=embedding_dim,
                    rnn_units=rnn_units,
                    batch_size=batch_size)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 128)           813952    
_________________________________________________________________
bidirectional (Bidirectional (64, None, 1024)          2625536   
_________________________________________________________________
bidirectional_1 (Bidirection (64, None, 1024)          6295552   
_________________________________________________________________
dense (Dense)                (64, None, 6359)          6517975   
Total params: 16,253,015
Trainable params: 16,253,015
Non-trainable params: 0
_________________________________________________________________


In [12]:
for input_example_batch, target_example_batch in seq_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 7, 6359)


In [13]:
# 选取所有的概率最大值的方式被称为贪心算法，这样的方式不一定能够得到整体概率最大值
# 我们使用随机采样的方式来获取
sample_indices = tf.random.categorical(logits=example_batch_predictions[0], num_samples=1)
print(sample_indices)
sample_indices = tf.squeeze(sample_indices, axis=-1)
print(repr(''.join(idx2_char[input_example_batch[0].numpy()])))
print(repr(''.join(idx2_char[target_example_batch[0].numpy()])))
print(''.join(idx2_char[sample_indices.numpy()]))

tf.Tensor(
[[ 941]
 [2325]
 [1396]
 [5798]
 [5294]
 [4177]
 [5087]], shape=(7, 1), dtype=int64)
'花木闲门苔藓生'
'浐川特去得吟情'
墓栲幅靓迈肖费


In [14]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(
        labels,
        logits,
        from_logits=True,
    )
model.compile(optimizer='adam', loss=loss)
example_loss = loss(target_example_batch, example_batch_predictions)
print(example_loss.shape, example_loss.numpy().mean())

(64, 7) 8.757623


In [15]:
output_dir = './text_generations_checkpoints'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
checkpoint_prefix = os.path.join(output_dir, 'check_{epoch}')
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only = True
)
epochs = 300
history = model.fit(seq_dataset, epochs = epochs, 
                    callbacks=[checkpoint_callback])

Epoch 1/300
1142/1142 [==============================] - 53s 47ms/step - loss: 6.8813
Epoch 2/300
 442/1142 [==========>...................] - ETA: 29s - loss: 6.5182

KeyboardInterrupt: 

## **3.4 载入模型与预测**

In [15]:
output_dir = './text_generations_checkpoints'
tf.train.latest_checkpoint(output_dir)

In [17]:
model2 = build_model(vocab_size, 
                     embedding_dim, 
                     rnn_units,
                     batch_size = 1)
model2.load_weights('./text_generations_checkpoints/check_600')
model2.build(tf.TensorShape([1, None])) # 加载后设置输入1歌样本变长序列

序列生成的流程应该是:
- 将a输入模型得到b
- 将ab输入到模型得到c
- 将abc输入到模型得到d
- 直到输出eos

In [18]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 128)            813952    
_________________________________________________________________
bidirectional_4 (Bidirection (1, None, 1024)           2625536   
_________________________________________________________________
bidirectional_5 (Bidirection (1, None, 1024)           6295552   
_________________________________________________________________
dense_2 (Dense)              (1, None, 6359)           6517975   
Total params: 16,253,015
Trainable params: 16,253,015
Non-trainable params: 0
_________________________________________________________________


In [19]:
def generate_text(model, start_string, generate_type = 'greedy'):
    input_eval = [char2idx[ch] for ch in start_string] # 一维
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    model.reset_states()
    predictions = model(input_eval)
    if generate_type == 'greedy':
        predictions_indices = tf.squeeze(tf.argmax(predictions, axis=-1), 0)
        print(predictions_indices)
        return ''.join(idx2_char[i] for i in predictions_indices)
    if generate_type == 'sample':
        predictions = tf.squeeze(predictions, 0)
        predictions_indices = tf.random.categorical(predictions, num_samples=1)
        return ''.join(idx2_char[i] for i in predictions_indices)    

In [23]:
text = '天长地久有时尽'
new_text_sample = generate_text(model2, text, generate_type='sample')
new_text_greedy = generate_text(model2, text, generate_type='greedy')
print(new_text_greedy, new_text_sample)

tf.Tensor([2536 1606 3999 3999 2087 3978 2201], shape=(7,), dtype=int64)
此恨绵绵无绝期 此恨人绵无绝期


C:\ProgramData\Anaconda3\envs\dp_gpu_tf2\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  from ipykernel import kernelapp as app
